In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
from zhipuai_llm import ZhipuAILLM
from dotenv import load_dotenv,find_dotenv
import os
_ = load_dotenv(find_dotenv())
api_key = os.environ["ZHIPUAI_API_KEY"]


In [3]:
zhipuai_model = ZhipuAILLM(model="glm-4-flash",temperature=0.1,api_key=api_key)
zhipuai_model("你好,自我介绍一下")

/data/tmp/ipykernel_38704/3987111452.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  zhipuai_model("你好,自我介绍一下")


'你好！我是一个人工智能助手，专门设计来帮助用户解答问题、提供信息、执行任务以及进行各种互动。我可以处理各种类型的问题，从日常咨询到复杂的数据分析，都能尽力提供帮助。无论何时何地，只要你有需要，都可以向我提问。很高兴认识你！有什么我可以帮你的吗？'

In [4]:
from zhipuai_embedding import ZhipuAIEmbeddings
from langchain.vectorstores.chroma import Chroma
embedding = ZhipuAIEmbeddings()
persist_directory = "./data_base/vector_db/chroma"
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)
print(f"向量库中存储的数量：{vectordb._collection.count()}")


/data/tmp/ipykernel_38704/3228571070.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from zhipuai_embedding import ZhipuAIEmbeddings
/data/tmp/ipykernel_38704/3228571070.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import C

向量库中存储的数量：20


In [5]:
question = "什么是prompt engineering"
docs = vectordb.similarity_search(question, 3)
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

    

检索到的第0个内容: 
 主题列表：美国航空航天局、当地政府、工程、员工满意度、联邦政府

给定文本: {story} """ response = get_completion(prompt) print(response) ```

[
  {"美国航空航天局": 1},
  {"当地政府": 1},
  {"工程": 0},
  {"员工满意度": 1},
  {"联邦政府": 1}
]

从输出结果来看，这个 story 与关于“美国航空航天局”、“员工满意度”、“联邦政府”、“当地政府”有关，而与“工程”无关。这种能力在机器学习领域被称为零样本（Zero-Shot）学习。这是因为我们并没有提供任何带标签的训练数据，仅凭 Prompt ，它便能判定哪些主题在新闻文章中被包含。

如果我们希望制定一个新闻提醒，我们同样可以运用这种处理新闻的流程。假设我对“美国航空航天局”的工作深感兴趣，那么你就可以构建一个如此的系统：每当出现与'美国宇航局'相关的新闻，系统就会输出提醒。
-----------------------------------------------------
检索到的第1个内容: 
 第五章 推断

在这一章中，我们将通过一个故事，引领你了解如何从产品评价和新闻文章中推导出情感和主题。

让我们先想象一下，你是一名初创公司的数据分析师，你的任务是从各种产品评论和新闻文章中提取出关键的情感和主题。这些任务包括了标签提取、实体提取、以及理解文本的情感等等。在传统的机器学习流程中，你需要收集标签化的数据集、训练模型、确定如何在云端部署模型并进行推断。尽管这种方式可能会产生不错的效果，但完成这一全流程需要耗费大量的时间和精力。而且，每一个任务，比如情感分析、实体提取等等，都需要训练和部署单独的模型。

然而，就在你准备投入繁重工作的时候，你发现了大型语言模型（LLM）。LLM 的一个明显优点是，对于许多这样的任务，你只需要编写一个 Prompt，就可以开始生成结果，大大减轻了你的工作负担。这个发现像是找到了一把神奇的钥匙，让应用程序开发的速度加快了许多。最令你兴奋的是，你可以仅仅使用一个模型和一个 API 来执行许多不同的任务，无需再纠结如何训练和部署许多不同的模型。

让我们开始这一章的学习，一起探索如何利用 LLM 加快我们的工作进程，提高我们

In [6]:
#构建检索回答链
#模板
from langchain.prompts import PromptTemplate
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)

In [7]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(zhipuai_model,
                                       retriever = vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

In [8]:
question_1 = "什么是南瓜书？"
question_2 = "王阳明是谁？"


In [9]:
result = qa_chain({"query":question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

/data/tmp/ipykernel_38704/3867639802.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query":question_1})


大模型+知识库后回答 question_1 的结果：
我不知道。谢谢你的提问！


In [10]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])


大模型+知识库后回答 question_2 的结果：
王阳明是中国明代著名的哲学家、军事家、教育家，也是心学的代表人物。他的思想对后世影响深远，尤其是在儒家哲学领域。谢谢你的提问！


In [11]:
zhipuai_model(question_1)

'南瓜书（Pumpkin Book）通常指的是一种儿童图书，其封面设计成南瓜的形状，寓意着收获和成长。这种书名或设计往往与秋季、南瓜节（如美国的南瓜节）等节日相关联。\n\n在中国，南瓜书可能特指一些以南瓜为主题的儿童读物，或者是封面设计成南瓜形状的儿童图书。这类书籍通常内容丰富，色彩鲜艳，旨在吸引小朋友们的兴趣，同时传递知识或教育意义。\n\n此外，“南瓜书”也可能是一个比喻，用来形容那些内容丰富、寓意深刻，但外表看起来并不起眼或平凡的书。这种说法强调了书籍内在价值的重要性。'

In [12]:
zhipuai_model(question_2)

'王阳明（1472年10月31日－1529年1月9日），原名王守仁，字伯安，号阳明，是明代著名的哲学家、军事家、教育家，也是心学的集大成者。他在中国哲学史上具有重要地位，被尊称为“阳明先生”。\n\n王阳明出生于浙江余姚，是明朝嘉靖年间的进士。他主张“知行合一”，认为知识和行动是不可分割的，强调通过实践来验证和实现知识。他的心学思想对后世影响深远，尤其是在伦理道德、政治哲学和教育思想等方面。\n\n在军事上，王阳明曾平定过多次叛乱，如平定南明永历帝的叛乱，被誉为“平定南明第一功臣”。在教育上，他提倡“格物致知”，强调通过观察事物来认识世界，培养人的道德品质。\n\n王阳明的思想在中国乃至东亚文化圈都有广泛的影响，被后人尊为“圣贤”。他的著作《传习录》等至今仍被广泛研究和传颂。'

In [13]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_message=True)

/data/tmp/ipykernel_38704/2734850115.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [20]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    zhipuai_model,
    retriever=retriever,
    memory=memory
)
question = "我可以学习到关于提示工程的知识吗？"
result = qa({"question": question})
print(result['answer'])


ValueError: Unsupported chat history format: <class 'str'>. Full chat history: Human: 我可以学习到关于提示工程的知识吗
AI: 是的，您可以学习到关于提示工程（Prompt Engineering）的知识。提示工程是利用自然语言处理（NLP）技术，特别是大型语言模型（LLM），来指导模型生成所需输出的过程。在上述文本中，提示工程被用来指导模型进行文本推理、情感分析、信息提取等任务。

以下是一些关于提示工程的关键点：

1. **任务指导**：通过编写特定的提示，您可以指导模型执行特定的任务，如情感分析、主题识别等。

2. **上下文提供**：提供上下文信息可以帮助模型更好地理解任务，并生成更准确的输出。

3. **结果验证**：通过分析模型的输出，您可以验证其是否满足需求，并据此调整提示。

4. **效率提升**：与传统的机器学习流程相比，提示工程可以显著提高工作效率，因为它减少了数据收集、模型训练和部署的步骤。

5. **适用性广**：提示工程适用于多种NLP任务，包括文本分类、情感分析、信息提取等。

通过学习提示工程，您可以更好地利用LLM的能力，解决实际问题，并提高工作效率。 

In [21]:
question = "为什么这门课需要教这方面的知识？"
result = qa({"question": question})
print(result['answer'])


ValueError: Unsupported chat history format: <class 'str'>. Full chat history: Human: 我可以学习到关于提示工程的知识吗
AI: 是的，您可以学习到关于提示工程（Prompt Engineering）的知识。提示工程是利用自然语言处理（NLP）技术，特别是大型语言模型（LLM），来指导模型生成所需输出的过程。在上述文本中，提示工程被用来指导模型进行文本推理、情感分析、信息提取等任务。

以下是一些关于提示工程的关键点：

1. **任务指导**：通过编写特定的提示，您可以指导模型执行特定的任务，如情感分析、主题识别等。

2. **上下文提供**：提供上下文信息可以帮助模型更好地理解任务，并生成更准确的输出。

3. **结果验证**：通过分析模型的输出，您可以验证其是否满足需求，并据此调整提示。

4. **效率提升**：与传统的机器学习流程相比，提示工程可以显著提高工作效率，因为它减少了数据收集、模型训练和部署的步骤。

5. **适用性广**：提示工程适用于多种NLP任务，包括文本分类、情感分析、信息提取等。

通过学习提示工程，您可以更好地利用LLM的能力，解决实际问题，并提高工作效率。 